In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:

# Load MovieLens Small dataset
#data = pd.read_csv("../../datasets/ml-latest-small/ratings.csv", sep=",")
data = pd.read_csv("../../datasets/ml-1m/ratings.dat", sep="::", names=["userId","movieId","rating","timestamp"])

#prefs = pd.read_csv("ratings.csv", sep=",")
#prefs = pd.read_csv("drive/MyDrive/PycharmProjects/datasets/ml-latest-small/ratings.csv", sep=",")

data.head()

/var/folders/8v/tw2k9h3n5k5d1stxdsh3q6d00000gn/T/ipykernel_34956/2819908162.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv("../../datasets/ml-1m/ratings.dat", sep="::", names=["userId","movieId","rating","timestamp"])


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
data.shape

(1000209, 4)

In [4]:
# Map user and movie IDs to unique indices
user_ids = data['userId'].unique()
movie_ids = data['movieId'].unique()

user_mapping = {user_id: idx for idx, user_id in enumerate(user_ids)}
movie_mapping = {movie_id: idx for idx, movie_id in enumerate(movie_ids)}

data['userId'] = data['userId'].map(user_mapping)
data['movieId'] = data['movieId'].map(movie_mapping)

In [5]:
class MovieLensDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe[['userId', 'movieId']].values
        self.ratings = dataframe['rating'].values.astype(np.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.LongTensor(self.data[idx]), torch.FloatTensor([self.ratings[idx]])


In [6]:
# Define the matrix factorization model
class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=20):
        super(MatrixFactorization, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        self.linear = nn.Linear(2*embedding_size, 1)

    def forward(self, X):
        user_embedding = self.user_embedding(X[:,0])
        movie_embedding = self.movie_embedding(X[:,1])
        #print("***********")
        #print(user_embedding)
        #print("***********")
        #print(movie_embedding)
        #print("***********")
        #a = torch.cat((user_embedding, movie_embedding), dim=1)
        #prediction = self.linear(a)
        #prediction = torch.sum(user_embedding + movie_embedding, dim=1)
        prediction = torch.sum(user_embedding * movie_embedding, dim=1)
        #print(prediction)
        #print("***********")

        return prediction

In [10]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.1)

# Initialize dataset and data loaders
train_dataset = MovieLensDataset(train_data)
test_dataset = MovieLensDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [17]:
a = torch.Tensor([[-0.4266, -0.4373, -0.4427, -0.7173,  0.1183],
        [ 0.5720,  1.3187, -1.5295, -1.7553,  0.9981]])
b = torch.Tensor([[-0.1013,  0.8980, -0.3187,  0.5583, -1.4790],
        [ 0.1501,  0.8112, -0.4382,  0.5182,  0.3780]])
torch.sum(a * b, dim=1)

tensor([-0.7838,  1.2935])

In [11]:
# Initialize the model and optimizer
num_users = len(user_ids)
num_movies = len(movie_ids)
model = MatrixFactorization(num_users, num_movies, embedding_size=10)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=1)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    absolute_errors = []

    for batch_idx, (X, y) in enumerate(train_loader):
        optimizer.zero_grad()
        predictions = model(X).squeeze()
        loss = criterion(predictions, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        absolute_errors.extend(torch.abs(predictions - y).tolist()[0])
        


    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")
    print(f"Train MAE: {np.mean(absolute_errors)}")



/Users/tevfikaytekin/miniforge3/envs/pytorch/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 

In [36]:
# Evaluation on the test set
model.eval()
test_loss = 0.0
absolute_errors = []
with torch.no_grad():
    for batch_idx, (X, y) in enumerate(test_loader):
        predictions = model(X).squeeze()
        loss = criterion(predictions, y)
        test_loss += loss.item()
        absolute_errors.extend(torch.abs(predictions - y).tolist()[0])

print(f"Test Loss: {test_loss / len(test_loader)}")
print(f"Test MAE: {np.mean(absolute_errors)}")

Test Loss: 1.0895969498006604
Test MAE: 0.862120256519469


/Users/tevfikaytekin/miniforge3/envs/pytorch/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([36, 1])) that is different to the input size (torch.Size([36])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
